# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code


In [1]:
# imports

import os
import io
import sys
import anthropic
import gradio as gr
import subprocess
import google.generativeai
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display


In [2]:
# Load environment variables from .env file
load_dotenv(override=True)

# API keys from environment
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
google_model = os.getenv('GOOGLE_MODEL')
ollama_base_url = os.getenv('OLLAMA_BASE_URL')
ollama_api_key = os.getenv('OLLAMA_API_KEY')
ollama_model = os.getenv('OLLAMA_MODEL', 'deepseek-v3.1:671b-cloud')

# Verify API keys
if openai_api_key:
    print(f"OpenAI API Key loaded: {openai_api_key[:8]}...")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key loaded: {anthropic_api_key[:7]}...")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key loaded: {google_api_key[:2]}...")
else:
    print("Google API Key not set")

if ollama_base_url:
    print(f"Ollama configured at: {ollama_base_url}")

OpenAI API Key loaded: sk-proj-...
Anthropic API Key loaded: sk-ant-...
Google API Key loaded: AI...
Ollama configured at: http://192.168.80.200:11434


In [3]:
# Initialize API clients
openai_client = OpenAI(api_key=openai_api_key)
claude_client = anthropic.Anthropic(api_key=anthropic_api_key)
genai = google.generativeai.configure(api_key=google_api_key)

# Initialize Ollama client (uses OpenAI-compatible API)
ollama_client = OpenAI(
    base_url=f"{ollama_base_url}/v1",
    api_key=ollama_api_key
)

print("All clients initialized successfully")

All clients initialized successfully


In [4]:
#OPENAI_MODEL = "gpt-5"
#CLAUDE_MODEL = "claude-sonnet-4-5-20250929"
#GROK_MODEL = "grok-4"
#GEMINI_MODEL = "gemini-2.5-pro"

# Want to keep costs ultra-low? Uncomment these lines:

OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-5-haiku-20241022"
GROK_MODEL = "grok-4-fast-non-reasoning"
GEMINI_MODEL = "gemini-2.5-flash-lite"

In [5]:
system_message = "Eres un asistente que reimplementa codigo python en C++ de alto rendimiento"
system_message += "para una maquina con sistema operativo Windows 11 Pro, Procesador 12th Gen Intel(R) Core(TM) i7-1255U (1.70 GHz)"
system_message += "Responde solo con codigo C++; no expliques tu trabajo mas alla de algunos comentarios"
system_message += "Presta atencion a los tipos de numeros para asegurar que no haya desbordamientos de int."
system_message += "La respuesta C++ debe producir una salida identica en el menor tiempo posible."

In [6]:
def user_prompt_for(python):
    user_prompt = "Reescribe este codigo python en C++ con la implementacion mas rapida posible que produzca una salida identica en el menor tiempo posible"
    user_prompt += "Responde solo con codigo C++; no expliques tu trabajo mas alla de algunos comentarios"
    user_prompt += "presta atencion a los tipos de numeros para asegurar que no haya desbordamientos de int (overflow)."
    user_prompt += "Recuerda incluir todos los paquetes de C++ necesarios, como iomanip"
    user_prompt += python
    return user_prompt

In [7]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [8]:
def write_output(cpp):
    # Limpia los marcadores de código markdown
    code = cpp.replace('```cpp','').replace('```','').strip()
    with open("optimized.cpp", "w", encoding="utf-8") as f:
        f.write(code)

In [9]:
def optimize_gpt(python):
    stream = openai_client.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [10]:
def optimize_claude(python):
    result = claude_client.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role":"user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [11]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [12]:
#exec(pi)

In [13]:
#optimize_gpt(pi)

In [14]:
# Compila C++ y corre el ejecutable (en el contenedor Linux de JupyterLab)
#!g++ -O3 -std=c++17 -o optimized optimized.cpp
#!./optimized

In [15]:
#optimize_claude(pi)

In [16]:
# Compila C++ y corre el ejecutable (en el contenedor Linux de JupyterLab)
#!g++ -O3 -std=c++17 -o optimized optimized.cpp
#!./optimized

In [17]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [18]:
# Version para compilar y ejecutar codigo C++ optimizado
def execute_cpp(code):
    write_output(code)
    try:
        compile_cmd = ["g++","-O3","-std=c++17","-o","optimized","optimized.cpp"]
        compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
        run_cmd = ["./optimized"]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"Ha ocurrido un error: \n{e.stderr}"


In [19]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

# Compiling C++ and executing

This next cell contains the command to compile a C++ file based on the instructions from GPT.

Again, it's not crucial to do this step if you don't wish to!

OR alternatively: student Sandeep K.G. points out that you can run Python and C++ code online to test it out that way. Thank you Sandeep!  
> Not an exact comparison but you can still get the idea of performance difference.  
> For example here: https://www.programiz.com/cpp-programming/online-compiler/

In [20]:
# Use the commands from GPT 5

#def compile_and_run():
#    subprocess.run(compile_command, check=True, text=True, capture_output=True)
#    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
#    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
#    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)

In [21]:
#compile_and_run()

In [22]:
#19.178207/0.082168

## OK let's try the other contenders!
### Example Definición del Generador Congruencial Lineal (LCG)

In [23]:
import time

# --- Definición del Generador Congruencial Lineal (LCG) ---
python_hard = """
def lcg(seed, a=1664525, c=1013904223, m=2**32):
    # Generador Congruencial Lineal (LCG) para generar números pseudoaleatorios.
    value = seed
    while True:
        value = (a * value + c) % m
        yield value

# --- Función para calcular la Suma Máxima del Subarray (Naive/Brute-Force) ---
def max_subarray_sum(n, seed, min_val, max_val):
    # Calcula la suma máxima del subarray en un arreglo de 'n' números
    # pseudoaleatorios generados por LCG, dentro del rango [min_val, max_val].
    # Utiliza el enfoque de fuerza bruta (O(n^2)).
    lcg_gen = lcg(seed)
    
    # Genera 'n' números pseudoaleatorios en el rango [min_val, max_val]
    # El módulo es (max_val - min_val + 1) para incluir max_val
    random_numbers = [(next(lcg_gen) % (max_val - min_val + 1)) + min_val for _ in range(n)]
    
    max_sum = float('-inf')  # Inicializa con menos infinito
    
    # Bucle de fuerza bruta O(n^2)
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
                
    return max_sum

# --- Función para ejecutar 'max_subarray_sum' 20 veces ---
def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    # Ejecuta 'max_subarray_sum' 20 veces, usando el LCG para obtener una
    # nueva semilla para cada corrida, y retorna la suma de todos los resultados.
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    
    for _ in range(20):
        # Obtiene una nueva semilla para la corrida actual
        seed = next(lcg_gen)
        # Calcula y acumula la suma máxima del subarray
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
        
    return total_sum

# --- Parámetros ---
n = 10000            # Número de random numbers (longitud del array)
initial_seed = 42    # Initial seed para el LCG principal
min_val = -10        # Minimum value of random numbers
max_val = 10         # Maximum value of random numbers

# --- Medición del tiempo de ejecución ---
print(f"Iniciando cálculo para N={n} y 20 corridas...")

start_time = time.time()

# Llama a la función principal que realiza las 20 corridas
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)

end_time = time.time()

# --- Resultados ---
print("-" * 40)
print(f"Total Maximum Subarray Sum (20 runs): {result}")
print(f"Execution Time: {end_time - start_time:.6f} seconds")
print("-" * 40)
"""

In [24]:
#exec(python_hard)

In [25]:
#optimize_gpt(python_hard)

In [26]:
# Compila C++ y corre el ejecutable (en el contenedor Linux de JupyterLab)
#!g++ -O3 -std=c++17 -o optimized optimized.cpp
#!./optimized

In [27]:
#optimize_claude(python_hard)

In [28]:
# Compila C++ y corre el ejecutable (en el contenedor Linux de JupyterLab)
#!g++ -O3 -std=c++17 -o optimized optimized.cpp
#!./optimized

## Generate Gradio

In [29]:
def stream_gpt(python):
    stream = openai_client.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [30]:
def stream_claude(python):
    result = claude_client.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role":"user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [31]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    else:
        raise ValueError("Modelo Desconocido")
    for stream_so_far in result:
        yield stream_so_far
        

In [ ]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("# Convierte codigo desde Python a C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=pi, lines=10)
        cpp = gr.Textbox(label="Code in C++:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT","Claude"], label="Selecciona model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convierte el codigo")
    with gr.Row():
        python_run = gr.Button("Execute Python")
        cpp_run = gr.Button ("Execute C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python Result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ Result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])
    
#ui.launch(inbrowser=True)
ui.launch(share=True, debug=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://29280b18c34b7add56.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
